# Autonomous Remote Controlled Car
-----
## Motivation
---
The only difference between a man and a machine is that a machine can perform a given task any number of times, with consistent accuracy. Having said that, we still fail to use a machine to its full potential. We still do chores that can very well be performed by a machine, and that too more efficiently. Driving a car is one such chore.
<p>
In india, every year, over 1,37,000 people are killed in road accidents. Most of these accidents occur due to human errors like drunk driving (not following traffic rules) or distracted driving. Traffic jams often render a driver frustrated, due to which the chances of reckless driving increases. In a country like Indiam where traffic congestion is a major issue, the technology of self-driving cars is inevitable. With the use of self-driving cars, road accidents and traffic jams can be reduced to a great extent, thus saving millions of lives, lakhs of rupees and a significant amount of time. This project is a small step towards this noble cause.

## The Idea
---
The project aims to modify a pre-built remote controlled car into a full fledged autonomous car with monocular vision capable of driving itself to a given destination safely and intelligently avoiding obstacles and minimizing the risk involved. The project uses multiple convolutional neural networks to analyse the environment and guide the car to the destination.

## The Plan
---
* A camera will be affixed to the top of a radio controlled car.
* A Raspberry Pi will be connected to a laptop computer (with a CUDA enabled GPU).
* The Raspberry Pi will be used only for relaying the data back and forth as it is not powerful enough to run deep convolutional neural networks.
* The Raspberry Pi will also be connected to the existing controller of the car which will be modified to enable the Raspberry Pi to digitally control the car.

The flow of data will occur as follows:<br>
The camera will capture images of the environment (at 30fps or greater) and will transmit the image frame to the computer via the Raspberry Pi.
The trained convolutional neural networks running on the computer will analyse the image in real time and produce one of four possible outputs: turn left, turn right, move forward or move backward.
This output will then be sent back to the Raspberry Pi which will relay it to the existing controller of the car.
In this way, we do not need to mount the RPi onto the car and can reduce its weight. But this limits the car to have a small control range. If we mount the RPi onto the car, we can achieve a greater control range but we cannot run heavy neural networks on it as a RPi cannot match the parallel processing power of a GPU. Nevertheless, we will try both the methods as there is a subtle tradeoff between performance and range. The communication channels between all the hardware parts can be set up in many different ways and Wi-Fi, bluetooth technologies can also be used.

## The Workflow
---
The workflow involves three major steps
#### 1. Establishing a connection
We will first set up all the connections as mentioned above and write a software API in Python that will relay keystrokes on the keyboard as outputs to the radio controlled car. This will effectively allow us to control the car from the laptop computer. Depending on the performance/range tradeoff we will choose which hardware setup suits the project best.
    
#### 2. Creating the model
After the hardware has been set up, we will set up the environment. Two different kinds of environments will be created. The first one will be an artificial environment where lane markers will be set up for the car to drive along. There will be traffic lights or 'Stop' signs where the car will be expected to stop until the light turns green. Obstacles will be set up along the path to simulate real world hindrances which the car will meander around after it is fully trained. The second environment will be the open world where there will be no lane markers or signs. The car will be given a destination or a general direction in which it should travel without colliding into obstacles. We will work on the first environment first. 
The machine learning pipeline will be as follows:

>#### a. Collecting data
Once we have built the environment, the camera will be fit onto the car and it will be run on the track several times to collect training data as images and their corresponding labels (the keypresses or direction in that particular frame). 

>#### b. Preprocessing
This training data will then be downsampled using OpenCV to a manageable number of pixels (say 60 x 80). The convolutional neural networks will not benefit from RGB images, so they will be converted to grayscale images to reduce dimensionality by three times. For the first attempt, basic sobel edge detection will be used to detect lanes and these processed images will be fed into the convolutional neural networks. If we get decent processing cycles (processing images atleast at 30Hz) we will continue with this approach. Otherwise, we will drop the edge detection algorithm as it is a costly process and feed the raw input images into the neural networks. If we still don't get decent feed rates, we will apply PCA (Principal Component Analysis) to the entire training dataset and try to reduce the dimensionality of the dataset and see if we get better results. For this, we will use the Scikit-learn library. Overall, we want to minimize the dimensions of the training set and also minimize the number of operations we do to preprocess the dataset, because we will also have to do the exact same operations on the test set in real time when we are testing our model. We would ideally like to keep the framerate within the 20-25fps range. Anything below 15fps will be pretty much useless.

>#### c. Creating and training the model
We will create the convolutional neural networks solely on Tensorflow as it is currently the most computationally efficient library. We might use third party wrappers like 'tflearn' or 'skflow' which provide higher levels of abstraction and simplify the model building process substantially. The first layer of the neural network will pretty much always be a 2d convolutional layer followed by max pooling and local response normalization. Further, we will add more convolutional layers and max pooling and try different permutations of layers and test them to see which gives the best results. The convolutional layers will be followed by one or two fully connected layers (depending on the sizes of the layers) and dropout layers will be used if the fully connected layers are too big. Rectified linear units will almost always be used as the activation function. The last layer will be a softmax classifier which will provide one of four possible outputs: Left, Right, Front or Reverse. This output information will then be sent to the Raspberry Pi through the RPi.GPiO module in Python which will communicate this as a direction to the car. For training, we plan to try both AdamOptimizer and Momentum optimizer to minimize the loss. The cross entropy function will be used as the loss function. Several versions of similar such models will be created on Tensorflow and tested to find the best possible model. We will also use existing models like Alexnet and test its performance. If we get a decent accuracy using any of these methods, we can create a Random Forest classifier and train several different models on different sections of the training set and work on a majority vote system during testing. The main disadvantage of this method is that it requires a lot of training data. Further, gradient boosting algorithms like AdaBoost will also be tried to see if performance increases. As a last resort, we will try 'Transfer Learning'. We will try to use the pretrained model 'Inception v3' and replace the last fully connected layer with our own layer trained on our own dataset and measure performance. We can also train Recurrent Neural Networks so that the car remembers previously seen environment configurations and locations. We expect that after finding the perfect model and training it on a particular track, the car will perform well even on previously unseen tracks. We are confident that atleast one of the above-mentioned methods will give us decent results. The car will also be trained to stop when it sees a 'Stop' sign. This will be implemented using the OpenCV computer vision library. For collision detection, we can help enhance the performance by using hardware addons like ultrasonic sensors. Since large neural networks take a long time to train on CPUs, we will use CUDA enabled GPUs (NVidia GPUs) to train our networks as Tensorflow is highly optimized for CUDA enabled GPUs and can massively reduce training time. This will speed up the workflow and enable us to try different possibilities. If CUDA enabled GPUs ar not available, we can use AMD GPUs with the Theano library, as Tensorflow does not support them.

>#### d. Testing the model
Before testing the model on the actual car, we will make an approximate simulation of the environment using the Unity3D game engine and the C# programming language. We will then train our neural networks on the simulation and measure performance. Even though this will not be an accurate representation of how the model will perform on the real car, we will get an intuition about which neural network cluster might perform better. We have already worked on most of the software part and we have clear ideas on how we will proceed with the project.

#### 3. Integration
After deciding which networks and algorithms to use, we will train these algorithms on the training set that we generated before, and then we need to set up all the connections and communication channels between the Raspberry Pi, the Pi camera, the computer and the car and integrate the hardware and software components of this project and test the networks on the car. After testing, we can fine tune the hyperparameters of the model (learning rate, dropout rate, number of layers, etc) to finally find the best algorithm for this task and reach the global optimum.



## Tech
---
<h3> Software </h3>
Python will be primarily used or creating, training and testing the neural networks. It will also be used for data collection and preprocessing the data. C# will be used along with the Unity3D game engine for testing our models on a simulated environment
The following python libraries will be used:
#### 1. Tensorflow
Tensorflow is an open source machine learning library by Google and most of the work will be done using this framework. It has a programmatic approach, contrasting the scripting approach of Python in general, that is, the code is not read line by line, but instead a computation graph is created and the entire model is run all at once. This approach of Tensorflow allows it to deliver very high performance which is invaluable for machine learning applications. Tensorflow also provides a graphing API called Tensorboard which can be run on a local server to graph the performance, loss, accuracy and other metrics related to the models during the training period.
#### 2. Scikit Learn
Scikit learn is also an open source machine learning library with high levels of abstraction. This will be used to quickly code up models and give us a starting point for the project. The cross validation and evaluation metrics provided in this package will be very useful to steer this project in the right direction and will help us decide which aspects of the project will be worth spending time on. This package will also be used in the preprocessing phase to analyse, cleanup and compress the training data and find correlations between features.
#### 3. OpenCV
OpenCV is an image recognition and computer vision library made in C++, but we will use a Python wrapper for it to make things simpler. Edge detection and interpolation algorithms will be used to detect lanes and edges and preprocess images for the convolutional neural network. OpenCV will also be used to reduce the dimensionality of the dataset. Camera input will be scaled down to a manageable number of pixels. The neural networks will not benefit from RGB images as we are using them to detect shapes and obstacles, so the input images will be converted to grayscale from RGB to reduce the size of the dataset by three times.
#### 4. NumPy
NumPy is a high performance numerical computation and linear algebra library built in C++. We will use this for matrix multiplications and speeding up computations in the preprocessing phase as NumPy arrays are substantially faster than native Python lists
#### 5. RPi.GPIO
The RPi.GPIO library allows us to easily configure and read-write the input/output pins on the Raspberry Pi's GPIO header within a Python script. The Raspbias OS will be used for the Pi as it has the best range of default softwares.
#### 6. PyautoGUI
The PyautoGUI library allows us to translate keystrokes on the keyboard as virtual keypresses. This will be useful when we want to test our models on the simulated environment, before testing on the actual car.

<h3> Hardware </h3>
#### 1. Raspberry Pi
Raspberry Pi will be primarily used as a communication device between the Pi camera and the Computer as the RPi is not powerful enough to train neural networks on.
#### 2. RF Camera
A camera that can communicate with the Raspberry Pi through radio waves. Pi camera can also be used to communicate with the laptop computer directly but then we would require a Wi-Fi 802.11n dongle to connect to RPi remotely and we would have to build a software API to set up the communication channels between them.
#### 3. Servo motor
To make the camera flexible to rotation
#### 4. Ultrasonic sensors
To detect obstacles outside of the field of view of the camera
#### 5. Radio controlled car
The pre-built car (which we already have) that we plan to automate